# Modeling Troubleshooting 

- baseline_model 이후
    - 모델 pipeline 구조 관련 의문점 해소
    - 모델 성능(f1-score) 향상 

--- 
### 📍 AttentionPooling 도입 이유 

- 문장에서 aspect-relevant 정보를 강조하려고
    - AttentionPooling은 문장 전체에서 aspect에 중요한 토큰들에만 집중해 문맥 벡터를 구성함
- [CLS]나 평균보다 유의미한 표현을 학습할 수 있어서
    - Transformer의 출력은 [batch, seq_len, hidden] 
    - [CLS] 벡터 : 항상 맨 앞에 있어서 aspect별 정보 반영이 약할 수 있음
    - 평균 풀링 : 모든 토큰을 동일하게 취급 -> 중요 단어가 묻힘
    - AttentionPooling :
        - 각 토큰의 중요도를 학습해서
        - 마지막 은닉 상태 hidden_states에서 중요한 토큰에 더 많은 가중치를 주고
        - weighted sum을 통해 하나의 벡터를 추출하기 때문에
        - aspect와 관련된 감정 신호를 더 잘 반영함
- Multi-aspect 상황에서 중요한 단어가 달라질 때 효과적
    - ABSA는 한 문장에서 여러 감정이 섞일 수 있기 때문에 단일한 [CLS] 벡터로는 감정 혼합 표현이 불리함
    - -> AttentionPooling은 문맥 내에서 동적으로 중요한 단어를 추출 가능 

#### hidden states란? 

- Transformer 모델의 출력 중 하나
- 입력 시퀀스의 각 토큰이 문맥 속에서 어떤 의미를 가지는지 벡터로 표현한 값
- 문장의 각 단어가 주변 단어들과 상호작용한 결과로 만들어진 토큰별 임베딩 벡터들
- hidden_states.shape = [batch_size, sequence_length, hidden_size]

#### hidden states가 필요한 이유 

- ABSA - 보통 문장 전체 표현이 필요한 ACD, ASC에 이 벡터를 활용
- Attention Pooling - 문장 전체에서 중요한 토큰에 주목하여 의미 벡터 생성
- [CLS] Pooling - 첫 번째 토큰(보통 문장 전체 요약 역할)의 벡터만 사용
- Mean Pooling - 모든 토큰의 평균을 문장 표현으로 사용
- Token-level task - 각 토큰별 분류
- Sentence-level task - 문장 전체 감정 분석

#### pooled outputd이란? 

- 문장 전체를 하나의 벡터로 요약한 표현
- 보통 Transformer 모델에서 [CLS] 토큰의 hidden state를 가공해서 만든 문장 레벨 표현

#### Electra에 pooled_out이 없는 이유 

- Electra는 BertModel과 구조가 비슷하지만 차이가 있음
    - Electra의 AutoModel은 outputs.last_hidden_state만 제공하고, BERT처럼 [CLS] 위치를 따로 분리한 pooled_output이 없음
        - 방법1 : [CLS]를 쓰려면 last_hidden_state[:, 0, :] 사용
        - 방법2 : AttentionPooling 사용
 
#### 우리 모델에서 AttentionPooling의 역할 

- 단순히 형식 맞추기가 아니라 실제로 유의미한 attention 기반의 가중합(pooling)을 수행하는 역할
- 단순히 [CLS] 토큰만 쓰는 대신 입력 시퀀스 전체에서 중요한 토큰에 가중치를 부여해서 문장 표현을 만들려고 한 것

#### AttentionPooling의 실제 작동 방식 

- 각 토큰에 대해 중요도를 학습된 방식으로 스코어링하고,
- attention_mask를 이용해 패딩 토큰은 제외하고,
- 가중치를 곱해 전체 시퀀스의 가중 평균을 계산함
- -> 단순 평균이나 [CLS]보다는 더 똑똑한 문장 요약을 만들 수 있다는 장점이 있음

#### AttetionPooling의 장점 

- 단순 평균이나 [CLS]보다 더 정보에 민감한 표현 가능
- 문장에서 중요한 토큰에 더 큰 가중치를 부여해서 성능 향상 기대

--- 
### 📍 GradScaler 도입 이유 

- 훈련 속도 향상
    - float16으로 연산하면 GPU에서 속도가 훨씬 빨라지고 VRAM도 덜 사용함 
    - -> 더 큰 배치 처리 가능
- 스케일링 & Underflow 방지
    - float16은 표현 가능한 숫자가 작아서 작은 gradient 값이 0으로 사라지는 현상(underflow) 발생
    - -> GradScaler가 자동으로 loss 값을 키워서 문제 방지

#### GradScaler란? 

- Mixed Precision Training (혼합 정밀도 학습)을 할 때 수치적으로 안정적인 학습을 가능하게 해주는 도구
- PyTorch에서 torch.cuda.amp.GradScaler로 제공됨
- 주로 autocast()와 함께 사용됨

#### autocast()란? 

- PyTorch에서 제공하는 자동 혼합 정밀도(Auto Mixed Precision) 기능
- float32 대신 일부 연산을 자동으로 float16으로 변환해 실행하는 컨텍스트 

--- 
### 📍 layer_norm 위치 

| 구조              | 사용 예               | 장점                                 |
|-------------------|------------------------|--------------------------------------|
| **Post-LN** Linear → Act → LN | BERT, RoBERTa 등       | 직관적 구조, shallow network에 안정 |
| **Pre-LN** LN → Linear → Act  | GPT-2, GPT-J, T5 등    | deep network에서 gradient 안정성 ↑<br>학습 초기 단계에서 성능 유지에 유리 |

- ABSA는 비교적 shallow architecture로 구성되어 있어서 Post-LN으로도 충분히 안정적이며 성능도 잘 나옴

--- 
### 📍 LayerNorm vs BatchNorm vs NoNorm 

- BERT 계열에서는 LayerNorm이 기본 (배치에 의존하지 않고 안정적)
- BatchNorm은 일반적으로 CNN 기반에서 사용됨
- Transformer에선 LayerNorm을 써야 안정적임 (BatchNorm은 피하는 게 좋음)

--- 
### 📍 annotation 없을 법한 문장 제거 후 predict 

- 평가 성능(F1 score)은 좋아질 수 있지만, 실제 의미 있는 성능 개선을 아닐 수 있음
- 데이터 분포의 왜곡
    - 원래 데이터에는 ‘속성 없음’ 문장이 일정 비율 포함되어 있음
    - 이를 제거하면 모델은 속성이 항상 존재하는 상황만 예측하게 되어 실제 사용 환경에서의 일반화 성능이 떨어질 수 있음
- 성능 과대 평가
    - 속성 없는 문장은 예측이 어려워 성능을 떨어뜨릴 수 있음
    - 이를 제거하면 결과적으로 쉬운 데이터만 predict 하게 되어 F1 score가 부풀려질 수 있음

#### 대안 : no aspect('없음') 클래스를 훈련시키지 않고 threshold로 처리

- 모델 설계 단순화
    - '없음'이라는 별도 클래스를 추가하지 않아도 됨 -> 클래스 수 고정
- 직관적인 제어 가능
    - threshold를 조절하여 예측 민감도를 직접 컨트롤 가능
- 노이즈 제거 가능
    - 불확실한 예측을 '없음'으로 버림으로써 precision 향상 가능

--- 
### 📍 threshold 설정 (last model 기준)

- ACD
    - min: 0.7700, mean: 0.9999, median: 1.0000
    - 거의 모든 예측의 confidence가 0.99 ~ 1.0 근처
- ASC
    - min: 0.5777, mean: 0.9967, median: 1.0000
    - 대부분의 예측 confidence가 매우 높지만, 간혹 0.6대인 예측도 있음

#### 결론 

- 대부분의 예측이 매우 confident하다
    - 거의 모든 예측이 softmax 확률 0.99 ~ 1.0
    - 모델이 거의 확신에 가까운 판단만 하고 있음
- threshold의 영향이 없을 가능성
    - 이미 대부분 예측이 매우 confident 하기 때문에 threshold가 filtering에 큰 영향을 끼치지 않을 수 있음
    - 따라서 threshold를 보수적으로 0.9 이상으로 설정
    - -> 낮은 confidence 예측만 제한적으로 filtering 하는 것이 적절하다고 판단 

--- 
### 📍 f1-score 계산 방식 (전체 vs 속성별)

| 항목               | evaluation_f1                                     | evaluation_per_aspect                        |
|--------------------|-----------------------------------------------------|------------------------------------------------|
| 평가 단위      | 전체 set 비교 (속성 또는 속성+감성 쌍)           | 각 aspect에 대한 binary 분류                   |
| 감성 포함 여부 | 포함 ((aspect, polarity) 쌍 기준 평가 가능)       | 미포함 (aspect 존재 여부만 평가)              |
| 정답 조건      | 완전 일치 (예측이 정답과 동일해야 TP로 인정됨)     | 해당 aspect 포함 여부만 확인                  |
| 계산 방식      | TP/FP/FN 직접 계산 후 수식으로 Precision/Recall/F1 | sklearn의 f1_score() 등 내장 함수 사용    |
| 출력 형태      | 전체 모델 성능 요약 (category + pipeline 2종류)    | 각 속성별 Precision / Recall / F1 / Support   |
| 사용 목적      | 모델 전반 성능 평가 (논문/대회 평가 기준)          | 세부 속성별 성능 분석 (어디서 약한지 확인)     |

--- 
### 📍 lable_smoothing 적용 

- 추가 클래스를 따로 정의할 필요 없이
- PyTorch 1.10 이후부터는 nn.CrossEntropyLoss에서 label_smoothing 인자를 공식적으로 지원함

#### lable_smoothing 이란? 

- 딥러닝 분류 문제에서 모델의 과신(overconfidence)을 줄이기 위한 정규화 기법
- 잘못된 예측에 대해 로스 급증을 완화함
- 노이즈가 있는 라벨에 대해 더 유연하게 학습함
- 테스트셋에서 성능 향상될 수 있음 (특히 불균형 클래스일 때 유효)

--- 
### 📍 활성화 함수 교체 

- GELU (Gaussian Error Linear Unit)
    - Tanh보다 부드러운 활성화 함수
    - BERT, RoBERTa, ELECTRA 등 많은 사전 학습 모델에서 기본으로 사용
    - 수식은 x * P(X ≤ x) 형태로 확률 분포 기반의 곡선 형태

#### 결과 : 성능 하락 

- ReLU는 일정 이하의 값을 0으로 날려 버려서 정제 역할을 해주는데, GELU는 그걸 안 해줘서 복잡도가 높아짐
- 음수 영역의 값도 일부 통과시켜서 모델이 더 많은 feature를 살리는 경향이 있음
- 데이터셋이 작거나 noisy한 feature가 많은 경우, 오히려 과적합으로 이어질 수 있음 

--- 
### 📍 Multi Dropout (stochastic dropout) 도입 

- 각 dropout이 랜덤하게 노드를 drop함 
- 여러 개의 dropout을 통과시켜서 평균을 취함 -> stochastic regularization 효과
- 일반적으로 ensemble과 비슷한 효과

#### 결과 : 성능 하락 

- MultiDropout은 일반적으로 대규모 데이터에서 성능 향상을 보여줌
- 데이터가 적거나 noise가 많은 경우에는 오히려 학습을 방해하는 요소가 될 수 있음

#### 뒤늦게 든 생각 

- Multi Dropout 도입 후 추론 시에도 model.train() 상태 유지가 필요한데
- 실수로 model.eval()에서 예측함 -> 그냥 평균이 의미 없는 예측이 됨
- 실제로 MultiDropout이 작동을 안 했던 듯 

--- 
### 📍 데이터 증강 기법 

| 기법 | 설명 | 장점 | 단점 |
|------|------|------|------|
| Back Translation (역번역) | 문장을 외국어로 번역 후 다시 원어로 번역 | - 문장 구조 다양화<br>- 의미 보존 잘 됨 | - 속성 단어 누락 위험<br>- 감정 왜곡 가능성 |
| Synonym Replacement (유의어 치환) | 감정 표현이나 속성을 유의어로 교체 | - 간단하고 빠름<br>- 레이블 유지 쉬움 | - 표현 다양성 부족<br>- 맥락 무시 가능성 |
| Template-based Generation | 템플릿에 속성과 감정을 끼워넣어 문장 생성 | - 레이블 정확도 높음<br>- 통제 가능 | - 문장 어색할 수 있음<br>- 다양성 낮음 |
| Conditional Generation | “속성=향, 감정=긍정” → LLM이 문장 생성 | - 표현 다양성 높음<br>- 유연한 활용 | - 제어 어려움<br>- 라벨 오류 가능성 |
| Random Insertion/Deletion | 감정 수식어를 임의로 삽입 또는 삭제 | - 간단<br>- 문장 자연도 유지 가능 | - 감정 세기 왜곡<br>- 의미 손상 가능 |
| MixUp / Interpolation | 둘 이상의 문장을 섞거나 연결 | - 과적합 방지<br>- 학습 일반화 유도 | - 문장 비자연적<br>- 감정/속성 모호성 |
| Entity Swap (속성 교체) | 동일 감정 문장에서 속성만 변경 | - 속성 다양화<br>- 감정-속성 분리 가능 | - 문맥 불일치 가능<br>- 레이블 검증 필요 |
| Paraphrasing | 문장 의미 유지하면서 표현만 바꿈 | - 다양성 증가<br>- 문장 자연스러움 | - LLM 필요<br>- 속성/감정 왜곡 가능성 |
| Weakly Supervised Labeling | 라벨 없는 문장에 약한 모델로 자동 라벨링 | - 데이터 확장에 유리 | - 라벨 정확도 낮음<br>- 학습 품질 저하 가능 |
| Style Transfer | 감정 스타일을 긍정↔부정으로 변경 | - 감정 다양화<br>- 실험적 활용 가능 | - 감정 오류 가능<br>- ABSA에선 라벨 위험성 ↑ |

#### Back Translation (역번역) 채택 

- 문장 구조를 다양화하면서 의미(속성·감정)을 유지할 수 있기 때문
    - 모델이 같은 의미의 다양한 표현을 학습 가능
- 노이즈가 적고 자동화하기 쉬운 증강 방식
    - 직접 문장을 쓰거나 템플릿을 설계하지 않아도 됨
    - 번역기만 있으면 대규모 자동 생성 가능
    - 감정 극성만 바뀌지 않게 하면 라벨 유지가 쉬움
- 감정 표현의 범위 확장에 효과적
    - 모델이 감정 표현에 과적합되지 않고 일반화된 감정 인식을 학습할 수 있음
- ABSA의 두 단계(ACD + ASC)에 모두 효과 있음
    - ACD (속성 인식) : 다양한 표현에서도 속성 감지 가능
    - ASC (감정 분류) : 같은 감정을 다양한 문장으로 표현해도 감정 극성 학습 가능

#### 중립/부정 감정 리뷰만 증강 

- 데이터 불균형 완화
- 모델이 부정/중립 감정을 잘 못 잡을 때가 많음
    - 부정 표현은 다양하고 은근함 (ex. "별로다", "추천은 안 해요", "기대보단 그저 그랬어요")
    - 중립 표현은 긍정/부정 중간이라 경계 모호 (ex. "괜찮았어요", "보통이에요", "그럭저럭")
    - -> 중립/부정을 증강하면 판단 기준이 더 명확해지고, 성능 개선됨
- F1 score 향상에 직결
    - 감정 분류는 보통 macro F1을 사용
    - 이는 모든 클래스의 F1을 평균하기 때문에 소수 클래스(중립/부정) 성능이 떨어지면 overall F1이 낮아짐
    - -> 중립/부정을 증강하면 전체 성능이 개선
- 데이터 증강이 긍정에선 오히려 역효과일 수 있음
    - 긍정 리뷰는 이미 많고 다양하므로 역번역으로 증강하면 중복·의미 훼손 위험 커짐 -> 모델에 noise 더하게 됨 

--- 
### 📍 StratifiedKFold 도입 

- 불균형한 데이터 분포 대응
    - 긍정 90%, 부정 10% 같은 데이터일 경우, 일반 KFold는 어떤 fold에 부정이 거의 없을 수 있음 -> 성능 왜곡
- 신뢰성 있는 평가
    - 각 fold의 클래스 분포가 비슷하므로 검증 성능이 더 일관되고 신뢰 가능함
- ABSA의 감정 분류(ASC)에서 특히 중요
    - 감정 클래스(positive/neutral/negative)의 분포가 한쪽으로 쏠려 있으면,
    - 모델이 긍정만 예측해도 높게 평가될 수 있음
- 증강 데이터 포함 시에도 클래스 균형 보장

#### StratifiedKFold란? 

- 분류 문제에서 클래스(레이블) 분포를 유지한 채 데이터를 K개로 나누는 교차 검증 방법
- 훈련/검증 데이터로 나눌 때 각 클래스(예: 긍정/부정/중립)의 비율이 원래 데이터와 비슷하게 유지되도록 분할함
- shuffle=True -> 데이터를 무작위로 섞고 나서 Fold를 나눔
- 하지만 random_state=1이 고정 -> 항상 같은 방식으로 섞여서 같은 Fold 순서가 보장됨
    - random_state는 StratifiedKFold 안에서만 작동함

#### set_seed(1, device) 

- 목적 : 폴드 분할의 순서를 항상 동일하게 유지하기 위해서
- 데이터 로딩 순서에 무작위성 있을 경우 무작위성 제어
- 모델 학습에서 재현성 유지
- 이후 다른 무작위 연산 대비
- -> set_seed()는 K-Fold 분할이 아닌 전체 실험 흐름의 무작위성 제어를 위한 것

--- 
### 📍 Spacing 도입 

#### 결과 : 성능 상승 

- Tokenizer가 올바르게 토큰 분리 가능
- Aspect 단어 인식이 쉬워짐
- Pretrained 모델 학습 환경과 맞춰짐
    - 많은 사전학습 언어모델들은 띄어쓰기 잘 된 데이터로 학습되어 있음
    - ex. 한국어 ELECTRA, KLUE-BERT 등은 뉴스, 위키 기반 데이터 사용
    - -> 입력 문장의 띄어쓰기가 잘 되어 있으면 사전학습된 언어모델의 표현력 활용이 극대화됨
- 감정 표현 단어가 더 뚜렷해짐

---
### 📍 max_len 조정 

- 3분위수(Q3) ≈ 95 -> 전체 문장 길이의 75%는 토큰 수가 95 이하
    - 모델 입력 토큰의 길이 분포(예: input_ids의 길이) 기준

#### max_len = 128 

- 충분한 커버리지 확보
    - Q3 = 95면 max_len=128은 대부분의 문장을 자르지 않고 처리 가능
    - 상위 25% 긴 문장 중 일부를 약간 잘라도 큰 정보 손실 없이 학습 가능
- Padding 낭비 최소화
- 모델 안정성 유지
    - 학습 중 너무 긴 입력은 메모리 문제, gradient exploding 위험 발생 가능
    - 128은 일반적으로 BERT 기반 모델에서 안정적인 길이로 간주됨

--- 
### sql db 구축 

- load_data.py 생성
- db.csv 생성
- (bash) python load_data.py 